In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import matplotlib.pyplot as plt
import numpy as np
import cv2


In [ ]:
# 1. Load MNIST Dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

print("Training samples:", x_train.shape)
print("Testing samples:", x_test.shape)


In [ ]:
# 2. IMAGE PROCESSING / PREPROCESSING

# Normalize to 0–1 range
x_train = x_train / 255.0
x_test = x_test / 255.0

In [ ]:
# Reshape for CNN input
# batch, height, width, channels(1 grayscale)
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)


In [ ]:
# 3. BUILD CNN MODEL
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2,2)),

    Conv2D(64, (3,3), activation='relu'), # deep feature detection
    MaxPooling2D((2,2)),   # reduces complexity

    Flatten(),  # converts 2D into 1D so that it can be passed to dense
    Dense(128, activation='relu'), # hidden neural layer decision making
    Dropout(0.3),  # reduces overfitting

    Dense(10, activation='softmax')  # give output from 0-9
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])  # adam - fast learning optimizer

model.summary()

In [ ]:
# 4. TRAIN MODEL
history = model.fit(
    x_train, y_train,
    epochs=5,
    validation_split=0.1
)  # learns to detect and train model

In [ ]:
# 5. EVALUATE MODEL

test_loss, test_acc = model.evaluate(x_test, y_test)
print("\nTest Accuracy:", test_acc)


In [ ]:
# 6. PREDICTION FROM TEST DATA

plt.figure(figsize=(12, 4))

for i in range(5):
    idx = np.random.randint(0, len(x_test))
    img = x_test[idx].reshape(28, 28)
    pred = model.predict(x_test[idx].reshape(1, 28, 28, 1))
    digit = np.argmax(pred)

    plt.subplot(1, 5, i+1)
    plt.imshow(img, cmap='gray')
    plt.title(f"Pred: {digit}")
    plt.axis("off")

plt.show()

In [ ]:
# 7. EXTERNAL IMAGE PREDICTION (REAL IMAGE)

def predict_external_image(image_path):
    # Read the image
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Check if image was loaded successfully
    if img is None:
        print(f"Error: Could not load image from {image_path}")
        return

    # Resize to 28x28
    img_resized_for_display = cv2.resize(img, (28, 28))

    # Invert colors if needed (white background, black digit)
    # This step is often crucial for hand-drawn digits where digits are dark on light background
    # but the model was trained on light digits on dark background (MNIST original format)
    img_inverted_for_model = cv2.bitwise_not(img_resized_for_display)

    # Normalize
    img_norm = img_inverted_for_model / 255.0

    # Reshape for model prediction
    final_img = img_norm.reshape(1, 28, 28, 1)

    # Predict
    pred = model.predict(final_img)
    digit = np.argmax(pred)

    print("Predicted digit:", digit)

    # Show the image as it's fed to the model (after inversion if applicable)
    plt.imshow(img_inverted_for_model, cmap='gray')
    plt.title(f"Predicted: {digit}")
    plt.axis('off')
    plt.show()

# Example (Only works if an image exists — you can upload in Colab):
predict_external_image("/content/Eight.jpg")

In [ ]:
# 8. SAVE MODEL

model.save("handwritten_digit_cnn_model.h5")
print("\nModel saved as handwritten_digit_cnn_model.h5")